In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import mlflow
from lib.norms import Norm1Scaler, Norm3Scaler
from lib.reproduction import major_oxides

In [2]:
norm = 3
drop_cols = major_oxides + ["ID", "Sample Name"]

In [3]:
import datetime

mlflow.set_experiment(f'GPR_Norm{norm}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')

2024/05/17 15:38:22 INFO mlflow.tracking.fluent: Experiment with name 'GPR_Norm3_20240517-153822' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/119', creation_time=1715953102128, experiment_id='119', last_update_time=1715953102128, lifecycle_stage='active', name='GPR_Norm3_20240517-153822', tags={}>

In [4]:
from experiments.optuna_run import get_data
from lib.cross_validation import get_cross_validation_metrics, perform_cross_validation
from lib.get_preprocess_fn import get_preprocess_fn
from lib.metrics import rmse_metric, std_dev_metric
from ngboost import NGBRegressor
import warnings

# disable warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

models = []


for target in major_oxides:
    folds, train, test = get_data(target)
    with mlflow.start_run(run_name=f"NGB_{target}"):
        # == Cross Validation ==
        scaler = Norm1Scaler() if norm == 1 else Norm3Scaler

        cv_metrics = perform_cross_validation(
            model=NGBRegressor(random_state=42),
            preprocess_fn=get_preprocess_fn(target_col=target, drop_cols=drop_cols, preprocessor=scaler),
            folds=folds,
            metric_fns=[rmse_metric, std_dev_metric]
        )
        
        mlflow.log_metrics(get_cross_validation_metrics(cv_metrics).as_dict())
        
        # == Training ==
        preprocess_fn = get_preprocess_fn(target_col=target, drop_cols=drop_cols, preprocessor=scaler)
        X_train, y_train, X_test, y_test = preprocess_fn(train, test)

        # Train the model
        model = NGBRegressor(random_state=42)
        model.fit(X_train, y_train)
        models.append(model)

        pred = model.predict(X_test)
        rmse = rmse_metric(y_test, pred)
        std_dev = std_dev_metric(y_test, pred)

        # Logging
        mlflow.log_params({"target": target, "norm": norm})
        mlflow.log_metrics({"rmse": rmse, "std_dev": std_dev})
        
        #mlflow.sklearn.log_model(ngbr, f"model_{target}")

SyntaxError: positional argument follows keyword argument (2560764418.py, line 21)